In [1]:
import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr
import datetime
from bs4 import BeautifulSoup

In [2]:
api_key = '133579488f1e7f9a44707b2a7d2be9999840935a'
dart = OpenDartReader(api_key)

In [3]:
day_buy = '2021-03-02'

In [4]:
list_d002 = dart.list(end=day_buy, kind='D', kind_detail='D002')
list_d002.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00608802,텔콘RF제약,200230,K,의결권대리행사권유참고서류,20210302003640,텔콘RF제약,20210302,
1,00240857,바이오스마트,038460,K,주식등의대량보유상황보고서(일반),20210302003619,박혜린,20210302,
2,00126308,삼성엔지니어링,028050,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210302003576,최춘기,20210302,
3,00110750,리더스 기술투자,019570,K,주식등의대량보유상황보고서(일반),20210302003573,리더스에셋홀딩스,20210302,
4,00765462,씨에스베어링,297090,K,의결권대리행사권유참고서류,20210302003565,씨에스베어링,20210302,


In [5]:
list_report_nm = list_d002[(list_d002['report_nm'] == '임원ㆍ주요주주특정증권등소유상황보고서') & ((list_d002['corp_cls'] == 'Y') | (list_d002['corp_cls'] == 'K' ))]
list_report_nm.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
2,00126308,삼성엔지니어링,028050,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210302003576,최춘기,20210302,
5,00954242,하이골드12호,172580,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210302003554,케이비증권,20210302,
6,00126308,삼성엔지니어링,028050,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210302003549,문덕규,20210302,
7,01368637,유일에너테크,340930,K,임원ㆍ주요주주특정증권등소유상황보고서,20210302003546,김이곤,20210302,
8,01368637,유일에너테크,340930,K,임원ㆍ주요주주특정증권등소유상황보고서,20210302003535,이강수,20210302,


In [6]:
result = pd.DataFrame(columns=['corp_name', 'corp_code', 'mdf_stk_cnt'])
for idx, row in list_report_nm.iterrows():
    html = dart.document(row['rcept_no'])
    soup = BeautifulSoup(html, 'html5lib')
    sec3_2 = soup.select('section-3')[2]
    rpt_rsn = sec3_2.findAll('tu', {'aunit': 'RPT_RSN'})[-1]
    mdf_stk_cnt = sec3_2.findAll('te', {'acode': 'MDF_STK_CNT'})[-1]
    if (rpt_rsn.contents[0] == '장내매수(+)'):
        tmp = pd.DataFrame(data=[[row['corp_name'], row['corp_code'], mdf_stk_cnt.contents[0]]], columns=['corp_name', 'corp_code', 'mdf_stk_cnt'])
        result = result.append(tmp, ignore_index=True)
result

,corp_name,corp_code,mdf_stk_cnt
0,하이골드12호,00954242,"4,931"
1,대림통상,00109824,"50,000"
2,유티아이,00961774,"5,500"
3,KTis,00372873,"1,680"
4,삼성화재해상보험,00139214,114
5,포스코케미칼,00155276,144
6,태영건설,00153861,"20,805"
7,웅진씽크빅,00628189,"3,906"
8,코아스,00210856,"64,258"


In [10]:
finstate = dart.finstate('00155276', 2020, reprt_code='11014')
finstate.set_index(['fs_nm','account_nm']).head()

rcept_no reprt_code bsns_year corp_code stock_code  \
fs_nm  account_nm                                                             
연결재무제표 유동자산        20201113000671      11014      2020  00155276     003670   
       비유동자산       20201113000671      11014      2020  00155276     003670   
       자산총계        20201113000671      11014      2020  00155276     003670   
       유동부채        20201113000671      11014      2020  00155276     003670   
       비유동부채       20201113000671      11014      2020  00155276     003670   

                  fs_div sj_div  sj_nm   thstrm_nm      thstrm_dt  \
fs_nm  account_nm                                                   
연결재무제표 유동자산          CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       비유동자산         CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       자산총계          CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       유동부채          CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       비유동부채         CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   

                       thstrm_amount frmtrm_nm      frmtrm_dt  \
fs_nm  account_nm                                               
연결재무제표 유동자산          906,122,359,371   제 49 기말  2019.12.31 현재   
       비유동자산       1,157,219,860,943   제 49 기말  2019.12.31 현재   
       자산총계        2,063,342,220,314   제 49 기말  2019.12.31 현재   
       유동부채          183,499,608,768   제 49 기말  2019.12.31 현재   
       비유동부채         869,915,888,037   제 49 기말  2019.12.31 현재   

                       frmtrm_amount ord thstrm_add_amount frmtrm_add_amount  
fs_nm  account_nm                                                             
연결재무제표 유동자산          649,644,634,524   1               NaN               NaN  
       비유동자산       1,080,487,930,571   3               NaN               NaN  
       자산총계        1,730,132,565,095   5               NaN               NaN  
       유동부채          250,654,203,754   7               NaN               NaN  
       비유동부채         473,160,418,105   9               NaN               NaN

In [8]:
# "11013"=1분기보고서,  "11012"=반기보고서,  "11014"=3분기보고서, "11011"=사업보고서
result2 = pd.DataFrame()
for idx, row in result.iterrows():
    finstate = dart.finstate(row['corp_code'], 2020, reprt_code='11014')
    if finstate is None:
        row['영업이익'] = "데이터없음"
        row['당기순이익'] = "데이터없음"
    elif finstate[finstate['fs_nm'] == '연결재무제표'].empty:
        row['영업이익'] = finstate.set_index(['fs_nm','account_nm']).loc[('재무제표', '영업이익'), 'thstrm_amount']
        row['당기순이익'] = finstate.set_index(['fs_nm','account_nm']).loc[('재무제표', '당기순이익'), 'thstrm_amount']
    else:
        row['영업이익'] = finstate.set_index(['fs_nm','account_nm']).loc[('연결재무제표', '영업이익'), 'thstrm_amount']
        row['당기순이익'] = finstate.set_index(['fs_nm','account_nm']).loc[('연결재무제표', '당기순이익'), 'thstrm_amount']
    result2 = result2.append(row)
result2

,corp_code,corp_name,mdf_stk_cnt,당기순이익,영업이익
0,00954242,하이골드12호,"4,931","-1,365,772,616","-1,365,815,270"
1,00109824,대림통상,"50,000","-3,187,661,424","-1,910,262,867"
2,00961774,유티아이,"5,500","341,001,143","860,994,656"
3,00372873,KTis,"1,680","2,946,591,968","3,270,788,060"
4,00139214,삼성화재해상보험,114,데이터없음,데이터없음
5,00155276,포스코케미칼,144,"6,852,279,605","19,380,619,218"
6,00153861,태영건설,"20,805","518,796,662,561","89,663,346,621"
7,00628189,웅진씽크빅,"3,906","9,396,739,209","8,885,390,919"
8,00210856,코아스,"64,258","-1,828,299,947","-2,083,378,623"
